In [1]:
from brt.cpp.router import generate_local_indices
import torch

dst_num = 4
gates = torch.randn((4, 4)).cuda()
topk_indices = torch.topk(gates, k=2, dim=1).indices

route_indices = (
    torch.zeros(
        gates.size(0),
        dst_num,
        dtype=torch.int32,
        device=gates.device,
    )
    .scatter_(1, topk_indices, 1)
    .cuda()
)

supported_capacities = torch.Tensor([1, 2, 4, 8, 16]).int().cuda()

local_indices, dst_loads = generate_local_indices(
    route_indices, supported_capacities
)
print(local_indices)
local_indices = local_indices * route_indices
print(local_indices)
print(dst_loads)




tensor([[0, 1, 1, 0],
        [1, 2, 1, 0],
        [2, 3, 1, 0],
        [3, 3, 1, 1]], device='cuda:0', dtype=torch.int32)
tensor([[0, 1, 1, 0],
        [1, 2, 0, 0],
        [2, 3, 0, 0],
        [3, 0, 0, 1]], device='cuda:0', dtype=torch.int32)
tensor([4, 4, 1, 1], device='cuda:0', dtype=torch.int32)


In [2]:
from brt.cpp.router import route_with_local_indices
import torch

in_data = torch.randn(4, 14, dtype=torch.float32).cuda()

total_load = torch.sum(dst_loads).item()
out_data = torch.zeros((total_load, 4), dtype=in_data.dtype).cuda()

new_gates = torch.ones_like(gates, dtype=torch.float32).cuda()

new_gates = new_gates

out_data = route_with_local_indices(
    in_data,
    local_indices,
    dst_loads,
    # new_gates,
)

print(in_data)
print(out_data)


tensor([[ 0.3469,  1.3397, -1.2007, -1.4127,  0.0895, -0.3332, -0.2247, -1.6396,
         -0.8053, -0.6212, -1.7722, -1.0833,  1.5094,  0.0053],
        [-1.3546,  0.2145, -1.9215,  1.5373, -0.3905, -0.8060, -0.1938,  0.5106,
         -1.8939, -0.5792,  0.6570,  0.2350,  0.4157, -0.3379],
        [ 1.0445, -0.4200, -0.6408, -0.7282,  0.8074,  0.8059, -0.2770, -2.2840,
         -0.4139,  0.4419,  0.6797, -0.4743, -0.2227,  0.1660],
        [-0.3661, -0.5215, -1.0101, -0.6265,  0.4868,  0.5627, -0.1698,  0.8852,
         -0.0618, -0.3311, -0.0492,  1.3408, -1.7260,  1.1847]],
       device='cuda:0')
tensor([[-1.3546,  0.2145, -1.9215,  1.5373, -0.3905, -0.8060, -0.1938,  0.5106,
         -1.8939, -0.5792,  0.6570,  0.2350,  0.4157, -0.3379],
        [ 1.0445, -0.4200, -0.6408, -0.7282,  0.8074,  0.8059, -0.2770, -2.2840,
         -0.4139,  0.4419,  0.6797, -0.4743, -0.2227,  0.1660],
        [-0.3661, -0.5215, -1.0101, -0.6265,  0.4868,  0.5627, -0.1698,  0.8852,
         -0.0618, -0.331